> To live! like a tree alone and free,
> and like a forest in solidarity...
> Nazim Hikmet

# Random Forests



__Objectives__

- Introduction of 'bagging' procedure.

- Identifying the need for bootstrapping for random forests

- Comparing Random forests and bagging methods

- Evaluating a model by random forest model

## Bootstrapping


<img src= "img/bootstrap1.png" style="height:250px">


# Bagging (Boostrapping + Aggregating)


Let's us one more time recall that if $Z_{1}, \cdots, Z_{n}$ are independent observations with variance $\sigma^{2}$ then the variance of the mean $\bar{Z}$ is given by $\frac{\sigma^{2}}{n}$. 

__How is this relevant now?__



We will use this idea calculate $$ \hat{f}^{1}(x), \cdots, \hat{f}^{B}(x)$$ where each $\hat{f}^{i}$ represents a decision tree fitted to the bootstrapped data.

Then we will make a prediction by: 

$$ \hat{f}_{\text{avg}}(x) = \frac{1}{B}\sum_{b=1}^{B} \hat{f}^{b}(x)$$

Note that this is for regression and for the classification we can get majority vote.

_side note: [sklearn averages over probabilities not majority vote](https://scikit-learn.org/stable/modules/ensemble.html#forest)_


## Sklearn for Random Forests

In [1]:
import pandas as pd
import numpy as np

In [2]:
# you can download the data from -- https://www.kaggle.com/ishaanv/ISLR-Auto#Heart.csv

# or http://faculty.marshall.usc.edu/gareth-james/ISL/data.html
heart = pd.read_csv('data/Heart.csv', index_col=0)
heart.head()
print(heart.shape)

(303, 14)


In [3]:
heart.dropna(axis=0, how='any', inplace=True)
y = heart.AHD
X = heart.drop(columns='AHD')

In [4]:
heart.head()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
1,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
2,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
3,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
4,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
5,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


In [5]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 1 to 302
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Age        297 non-null    int64  
 1   Sex        297 non-null    int64  
 2   ChestPain  297 non-null    object 
 3   RestBP     297 non-null    int64  
 4   Chol       297 non-null    int64  
 5   Fbs        297 non-null    int64  
 6   RestECG    297 non-null    int64  
 7   MaxHR      297 non-null    int64  
 8   ExAng      297 non-null    int64  
 9   Oldpeak    297 non-null    float64
 10  Slope      297 non-null    int64  
 11  Ca         297 non-null    float64
 12  Thal       297 non-null    object 
 13  AHD        297 non-null    object 
dtypes: float64(2), int64(9), object(3)
memory usage: 34.8+ KB


In [28]:
display(X.Thal.value_counts(normalize = True))
display(X.ChestPain.value_counts(normalize = True))

normal        0.552189
reversable    0.387205
fixed         0.060606
Name: Thal, dtype: float64

asymptomatic    0.478114
nonanginal      0.279461
nontypical      0.164983
typical         0.077441
Name: ChestPain, dtype: float64

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 1, stratify=y)

In [30]:
from sklearn.preprocessing import OneHotEncoder

In [37]:
categorical_variables = list(X_train.select_dtypes(include=['object']).columns)
numerical_variables = list(X_train.select_dtypes(include = ['int64', 'float64']).columns)

In [41]:
categorical_variables.append(numerical_variables.pop(-2))

In [42]:
categorical_variables 

['ChestPain', 'Thal', 'RestECG', 'Slope']

In [43]:
numerical_variables

['Age', 'Sex', 'RestBP', 'Chol', 'Fbs', 'MaxHR', 'ExAng', 'Oldpeak', 'Ca']

In [44]:
ohe = OneHotEncoder(drop='first')
X_categ = ohe.fit_transform(X_train[categorical_variables]).toarray()
X_num = X_train[numerical_variables].values
Xtrain = np.concatenate((X_categ, X_num), axis=-1,)
Xtrain.shape

(237, 18)

In [45]:
# now we should transform the test data
# to be able to use it for the prediction

X_test_categ = ohe.transform(X_test[categorical_variables]).toarray()
X_test_num = X_test[numerical_variables].values
Xtest = np.concatenate((X_test_categ, X_test_num), axis=-1,)
Xtest.shape

(60, 18)

In [46]:
from sklearn.ensemble import RandomForestClassifier

In [47]:
clf = RandomForestClassifier(n_estimators=100,
                             criterion='gini',
                             max_features='auto',
                             oob_score=True)

In [48]:
clf.fit(Xtrain, y_train)
print(clf.score(Xtrain, y_train))
print(clf.score(Xtest, y_test))

1.0
0.8


__Your Turn__

- Use 5 fold cross_validation to fit random forest classifier we created above.
- Don't forget to return training scores and trained estimators.

In [49]:
from sklearn.model_selection import cross_validate

In [51]:
cv = cross_validate(clf, Xtrain, y_train, return_estimator=True, return_train_score=True)

__Your Turn__

- What is the type of validator above?

- Check test vs train(validation) scores.

- Print "mean +/- std" for both train and test scores

- Also print oob_scores and compare them with cross_validation scores

In [52]:
type(cv)

dict

In [53]:
cv

{'fit_time': array([0.18010592, 0.15965414, 0.15572381, 0.1588757 , 0.16227674]),
 'score_time': array([0.0100739 , 0.00833368, 0.00807118, 0.00927424, 0.00873399]),
 'estimator': (RandomForestClassifier(oob_score=True),
  RandomForestClassifier(oob_score=True),
  RandomForestClassifier(oob_score=True),
  RandomForestClassifier(oob_score=True),
  RandomForestClassifier(oob_score=True)),
 'test_score': array([0.79166667, 0.8125    , 0.78723404, 0.76595745, 0.85106383]),
 'train_score': array([1., 1., 1., 1., 1.])}

In [54]:
cv['test_score'].mean()

0.8016843971631206

In [70]:
clf.fit(Xtrain, y_train)
clf.oob_score_

0.8016877637130801

__Your Turn__

- Note that we have over-fitting problem. 

- Let's try to reduce over-fitting

In [ ]:
y_train.value_counts()

In [56]:
clf = RandomForestClassifier(max_depth=10,
                             max_features = 'log2',
                             oob_score=True)

clf.fit(Xtrain, y_train)
print(clf.score(Xtrain, y_train))
print(clf.oob_score_)

1.0
0.7763713080168776


### Do it with Pipelines!

In [77]:
# There is an "easier" way to do this
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, plot_roc_curve

In [89]:
numeric_features = numerical_variables 
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_features = categorical_variables 
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(n_estimators=100,
                             criterion='gini',
                             max_features='auto',
                             oob_score=True))]) # penalty = 'l2', class_weight = 'balanced'

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf.fit(X_train, y_train)
print("model traing score: %.3f" % clf.score(X_train, y_train))
print("model test score: %.3f" % clf.score(X_test, y_test))

model traing score: 1.000
model test score: 0.783


In [90]:
train_pred = clf.predict_proba(X_train)
test_pred = clf.predict_proba(X_test)
train_score = roc_auc_score(y_train, train_pred[:,1])
test_score = roc_auc_score(y_test, test_pred[:,1])

print(f'Train ROC-AUC score:{train_score}')
print(f'Test ROC-AUC score:{test_score}')

Train ROC-AUC score:1.0
Test ROC-AUC score:0.8487723214285714


In [91]:
pipe_validator = cross_validate(clf, X_train, y_train, return_estimator=True, return_train_score=True)

In [92]:
cv

{'fit_time': array([0.18010592, 0.15965414, 0.15572381, 0.1588757 , 0.16227674]),
 'score_time': array([0.0100739 , 0.00833368, 0.00807118, 0.00927424, 0.00873399]),
 'estimator': (RandomForestClassifier(oob_score=True),
  RandomForestClassifier(oob_score=True),
  RandomForestClassifier(oob_score=True),
  RandomForestClassifier(oob_score=True),
  RandomForestClassifier(oob_score=True)),
 'test_score': array([0.79166667, 0.8125    , 0.78723404, 0.76595745, 0.85106383]),
 'train_score': array([1., 1., 1., 1., 1.])}

In [93]:
# train scores
print(cv['train_score'])
# validation scores
print(cv['test_score'])

# let's pick one of the estimator for further investigation

est = cv['estimator'][0]

[1. 1. 1. 1. 1.]
[0.79166667 0.8125     0.78723404 0.76595745 0.85106383]


In [94]:
cv['test_score'].mean()

0.8016843971631206

In [95]:
est

RandomForestClassifier(oob_score=True)

In [97]:
est['classifier'].oob_score_

TypeError: list indices must be integers or slices, not str

## Feature Importance

In [61]:
feature_importances = clf.feature_importances_

In [62]:
feature_importances

array([0.03729048, 0.01519941, 0.01780007, 0.11698337, 0.09956156,
       0.000284  , 0.02122596, 0.02899576, 0.00381653, 0.09562926,
       0.03305331, 0.08508576, 0.08426948, 0.01201614, 0.10070214,
       0.04479673, 0.10932098, 0.09396907])

In [64]:
# be careful with the order of columns
columns = ohe.get_feature_names().tolist() + numerical_variables

In [65]:
importances = pd.DataFrame(data=feature_importances,
                           index=columns, columns=['feature_importances'])

importances.sort_values(by='feature_importances', ascending=False)

,feature_importances
x1_normal,0.116983
Oldpeak,0.109321
MaxHR,0.100702
x1_reversable,0.099562
Age,0.095629
Ca,0.093969
RestBP,0.085086
Chol,0.084269
ExAng,0.044797
x0_nonanginal,0.037290


### Extra Material 

- [Sklearn averages probabilities in RF implementation](https://scikit-learn.org/stable/modules/ensemble.html#forest)

- [On the variance](https://newonlinecourses.science.psu.edu/stat414/node/167/)

- [Is RF immune to overfitting?](https://en.wikipedia.org/wiki/Talk%3ARandom_forest)

- [Tricky stuff with respect to feature importance](http://rnowling.github.io/machine/learning/2015/08/10/random-forest-bias.html)

- [An interesting implementation of feature importance](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances_faces.html#sphx-glr-auto-examples-ensemble-plot-forest-importances-faces-py)

- [Different Ensemble Methods in sklearn](https://scikit-learn.org/stable/modules/ensemble.html#forest)

- [ISLR - section 8.2](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf)

- [Another library for RF: H2o](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/drf.html)